# OpenCV初探

OpenCV 是计算机视觉领域常用的工具。OpenCV 功能强大内容丰富，很多CV任务都可以使用 OpenCV 完成  
本次课程主要是展示 openCV 的常用用法，为后续实验提供必要支持  

本节内容主要包含：  
1. 导入cv库
2. 读取图片并展示
3. 保存图片
4. 缩放图片
5. 剪切图片
6. 镜像图片
7. 旋转图片


<font color="red">注意</font>解压图片文件，解压语句只需在第一次执行的时候执行一次，之后可以加注释，使其不再执行

In [1]:
#数据集解压
#!unzip -oqj '/home/aistudio/data/data54426/CV常用工具_OpenCV初探_专用数据集.zip' -d './data'

## 1. 导入cv库
OpenCV 支持各平台和多种语言。由于目前 AI 领域Python语言使用较多，本次课程主要使用 Python 中的 OpenCV 类库。  

Numpy 是 Python 的一个数学库，是很多 AI 类库的底层类库。  
Matplotib 是 python 的一个绘图库，其内置了各种各样的绘图方法，是做 AI 可视化、AI 调试的主要工具  

> 这里没有安装过程，是因为环境中已经预先安装好了

In [6]:
#导入 类库
import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

#查看opencv的版本
print(cv2.__version__)

## 2.读取图片并展示
读取磁盘文件到内容是 OpenCV 进行图片操作的第一步。（如果图片本身已经在内存中可以省略这一步）  
将内存中的图片显示出来，也是 OpenCV 最基础的操作，也是做 AI 视觉时候在调试阶段常用的操作。

> 如果是在本地而不是在云上（或者说不是在浏览器中），那么显示图片使用的是 cv2.imshow 函数

In [7]:
###    读取磁盘图片文件    ###
#通过imread函数，读取图片；
#    imread函数有两个参数，第一个参数是图片路径，第二个参数表示读取图片的形式：
#        cv2.IMREAD_COLOR：加载彩色图片，这个是默认参数，可以直接写1。
#        cv2.IMREAD_GRAYSCALE：以灰度模式加载图片，可以直接写0。
#        cv2.IMREAD_UNCHANGED：包括alpha，可以直接写-1
img = cv2.imread('./data/fox.jpg',1)

###    将图片显示出来    ###
plt.imshow(img)

观察上图可以发现，该图与原图色彩是不同（一只红色的狐狸变成了蓝色）。这是因为我们读取图片使用的是 openCV 而显示图片 使用的是 matplotlib 库（简写作 plt ）。使用 plt 显示时，颜色偏差严重的情况。这是因为plt和imread二者颜色通道不同导致。

openCV 的 imread 函数读取图片的时候认为色道顺序是 BRG ，而 plt.imshow 函数显示图片时认为图片的色道是 rgb 。    
找到了问题的原因了，也就知道如何修改了。只需要将 内存中的矩阵的 B 通道放到最后一个位置，R和G 通道迁移即可。


具体修改方式有两种：
1. 先分离，再合并
```
b, g, r = cv2.split(img)
img2 = cv2.merge([r,g,b])
```
2. 使用语法技巧  
	`img2 = img[:,:,::-1]`
3. 使用 OpenCV 内置函数  
	`img2 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)`



In [8]:

b, g, r = cv2.split(img)
img2 = cv2.merge([r,g,b])

# img2 = img[:,:,::-1] 

# img2 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.imshow(img2)

### 练习
读取图片（路径为"./data/snow_spanther.jpg"）并且显示在页面上

In [9]:
import cv2
from matplotlib import pyplot as plt

#-------- 请开始编写代码
my_img = cv2.imread("./data/snow_panther.jpg", 1)
b, g, r = cv2.split(my_img)
my_img2 = cv2.merge([r, g, b])
# plt.imshow(my_img)
plt.imshow(my_img2)
#-------- 请结束编写代码

## 3.保存图片
把在内存中修改过的图片的数据保存到硬盘上也是一个基本操作。
请运行以下代码，然后在对应路径下查看是否生成了对应的图片。

>cv2.imwrite的参数   
>cv2.CV_IMWRITE_JPEG_QUALITY  设置图片格式为.jpeg或者.jpg的图片质量，其值为0---100（数值越大质量越高），默认95  
>cv2.CV_IMWRITE_WEBP_QUALITY  设置图片的格式为.webp格式的图片质量，值为0--100  
>cv2.CV_IMWRITE_PNG_COMPRESSION  设置.png格式的压缩比，其值为0--9（数值越大，压缩比越大），默认为3  
`cv2.imwrite('./data/fox_copy.jpg',img,[cv2.IMWRITE_JPEG_QUALITY,50])`

In [10]:
import cv2
img = cv2.imread('./data/fox.jpg',1)

#cv2.imwrite表示保存图像，
#第一个参数是保存的图像的路径，第二个参数是要保存的图像。
cv2.imwrite('./data/fox_copy.png',img) 

### 练习
读取图片（路径为"./data/snow_panther.jpg"）将其保存为 "./data/snow_panther_copy.jpg"

In [11]:
import cv2

#-------- 请开始编写代码
img = cv2.imread('./data/snow_panther.jpg',1)
cv2.imwrite('./data/snow_spanther_copy.png',img) 
#-------- 请结束编写代码

## 4.缩放图片
图片的缩放也是基本操作，在某些深度学习理论中会使用到图像金字塔的概念，即 图像不断缩放形成 由小到大的 一系列图片然后分别送入网络。  

<img src="./data/Image_Pyramid.jpg" width = "300" height= "300" align="center" />


下面首先打印原图，然后分别打印 1/2 大小图片 和 1/4 大小图片
观察图片的相关信息，获取图片的 宽 和 高 

In [12]:
import cv2
from matplotlib import pyplot as plt

#读取图片
tiger_original = cv2.imread('./data/tiger.jpg',1)

#读出的图片信息是多维矩阵
#我们可以把这个矩阵信息打印出来
imgInfo = tiger_original.shape
height = imgInfo[0]
width = imgInfo[1]
channel = imgInfo[2]
print(imgInfo)
print(height)
print(width)
print(channel)

#打印图片
tiger_original = tiger_original[:,:,::-1] 
plt.imshow(tiger_original)

### cv2.resize 函数
缩放图片使用 cv2.resize 函数。

### 函数解析：
##### cv2.resize(InputArray src, OutputArray dst, Size, fx, fy, interpolation)

|  参数        | 含义      |
|  :----        | :----  |
| InputArray src  | 输入图片 |
| OutputArray dst  | 输出图片 |
| Size         | 输出图片尺寸 |
| fx, fy       |沿x轴，y轴的缩放系数 |
| interpolation  |插入方式 |
 

<font face="黑体" color=red size=3>特别注意：使用cv2.resize()时，size参数顺序是 **宽×高**</font>


其中，interpolation表示插入方式，有以下几种方式：

|  参数        | 含义      |
|  :----        | :----  |
| INTER_NEAREST | 	最近邻插值 |
| INTER_LINEAR  | 双线性插值（默认设置） |
| INTER_AREA         | 使用像素区域关系进行重采样 |
| INTER_CUBIC      |4x4像素邻域的双三次插值 |
| INTER_LANCZOS4  |8x8像素邻域的Lanczos插值 |


#### 缩放为原来的 1/2 

In [13]:
import cv2
from matplotlib import pyplot as plt

tiger_original = cv2.imread('./data/tiger.jpg',1)
imgInfo = tiger_original.shape
print(imgInfo)
height = imgInfo[0]
width = imgInfo[1]
mode = imgInfo[2]

# 我们把原图片进行等比例缩放 
dstHeight = int(height*0.5)
dstWidth = int(width*0.5)

# 缩放到原来的二分之一，输出尺寸格式为（宽，高）
tiger_resized_half = cv2.resize(tiger_original,(dstWidth,dstHeight))

# 显示图片
tiger_resized_half_show = tiger_resized_half[:, :, ::-1]
plt.imshow(tiger_resized_half_show)

#保存
cv2.imwrite('./data/tiger_resized_half.jpg',tiger_resized_half)


#### 缩放为原来的 1/4
用另一种语法实现

In [15]:
import cv2
from matplotlib import pyplot as plt

tiger_original = cv2.imread('./data/tiger.jpg',1)
imgInfo = tiger_original.shape
print(imgInfo)
height = imgInfo[0]
width = imgInfo[1]
mode = imgInfo[2]

# 我们把原图片进行等比例缩放 
dstHeight = int(height*0.25)
dstWidth = int(width*0.25)

# 缩放到原来的四分之一，输出尺寸格式为（宽，高）
# tiger_resized_quarter = cv2.resize(tiger_original,(dstWidth,dstHeight))
tiger_resized_quarter = cv2.resize(tiger_original, (0, 0), fx=0.25, fy=0.25, interpolation=cv2.INTER_NEAREST)

# 显示图片
tiger_resized_quarter_show = tiger_resized_quarter[:, :, ::-1]
plt.imshow(tiger_resized_quarter_show)

#保存
cv2.imwrite('./data/tiger_resized_quarter.jpg',tiger_resized_quarter)

### 练习
请将图片 './data/tiger' 缩放为原来图片的 1/8 
显示出来，并且保存到硬盘上


In [16]:
import cv2
from matplotlib import pyplot as plt

#-------- 请开始编写代码
tiger_original = cv2.imread('./data/tiger.jpg',1)
imgInfo = tiger_original.shape
print(imgInfo)
height = imgInfo[0]
width = imgInfo[1]
mode = imgInfo[2]

# 我们把原图片进行等比例缩放 
dstHeight = int(height*0.125)
dstWidth = int(width*0.125)

# 缩放到原来的八分之一，输出尺寸格式为（宽，高）
tiger_resized_eighth = cv2.resize(tiger_original,(dstWidth,dstHeight))

# 显示图片
tiger_resized_eighth_show = tiger_resized_eighth[:, :, ::-1]
plt.imshow(tiger_resized_eighth_show)

#保存
cv2.imwrite('./data/tiger_resized_eighth.jpg',tiger_resized_eighth)
#-------- 请开结束编写代码

## 5. 图片剪切
在进行数据集的数据扩充的时候，通常会使用 截取图片 80% 的区域然后在恢复大小的方法。  
那么下面来看看如何截取图像的一部分。


In [17]:
import cv2
from matplotlib import pyplot as plt

img_original = cv2.imread('./data/tiger.jpg',1)

#获得图片的形状
imgInfo = img_original.shape
print(imgInfo)
height = imgInfo[0]
width = imgInfo[1]
mode = imgInfo[2]

#截取原图的一部分
#参数1 是高度的范围，参数2是宽度的范围
img_cropped = img_original[200:1000,400:800]

#显示图片
img_cropped_show = img_cropped[:, :, ::-1]
plt.imshow(img_cropped_show)

#保存
cv2.imwrite("./data/img_cropped.jpg",img_cropped)


### 练习 
请随机的截取出一个图片，该图片的高度是原图片的 80%，该图片的宽度是原图片的 80%
将该图片显示出来，并且保存到硬盘上，新图片命名为 "img_cropped_0.8.jpg"

#### 提示
Python 产生随机数使用 random 包。该场景下可以考虑使用 uniform 函数，其基本用法如下：
```
import random
rand = random.uniform(0, 0.2)
print(rand)
```

In [18]:
import cv2
from matplotlib import pyplot as plt
import random

#-------- 请开始编写代码
tiger_original = cv2.imread('./data/tiger.jpg',1)
imgInfo = tiger_original.shape
print(imgInfo)
height = imgInfo[0]
width = imgInfo[1]
mode = imgInfo[2]

# 确定截图图片的开始位置
random_start = random.uniform(0, 0.2)
dstHeight_start = int(height*random_start)
random_start = random.uniform(0, 0.2)
dstWidth_start = int(width*random_start)

# 确定截图图片的结束位置
dstHeight_end = int(dstHeight_start+height*0.8)
dstWidth_end = int(dstWidth_start+width*0.8)
#截取原图的一部分
#参数1 是高度的范围，参数2 是宽度的范围
img_cropped_temp = tiger_original[dstHeight_start:dstHeight_end , dstWidth_start:dstWidth_end ]

#显示图片
img_cropped_show = img_cropped_temp[:, :, ::-1]
plt.imshow(img_cropped_show)

#保存
cv2.imwrite("./data/img_cropped_temp.jpg",img_cropped_temp)
#-------- 请结束编写代码

## 6.镜像图片

在进行数据集的数据扩充的时候，通常会使用 图片镜像的方法增加数据。
那么下面来看看如何做图片镜像。

In [19]:
import cv2
from matplotlib import pyplot as plt

bird_original=cv2.imread("./data/bird.jpg")
#显示图片
bird_original = bird_original[:, :, ::-1]
plt.imshow(bird_original)

#水平镜像
bird_flip_Horizontally=cv2.flip(bird_original,1)

#垂直镜像
bird_flip_Vertically=cv2.flip(bird_original,0)

#水平垂直镜像
bird_flip_Horizontally_Vertically=cv2.flip(bird_original,-1)

#保存图片
cv2.imwrite("./data/bird_flip_Horizontally.png",bird_flip_Horizontally)
cv2.imwrite("./data/bird_flip_Vertically.png",bird_flip_Vertically)
cv2.imwrite("./data/bird_flip_Horizontally_Vertically.png",bird_flip_Horizontally_Vertically)

### 练习
对老虎图片做 水平镜像、垂直镜像、水平垂直镜像

In [20]:
import cv2
from matplotlib import pyplot as plt

#-------- 请开始编写代码
tiger_original=cv2.imread("./data/tiger.jpg")
#显示图片
tiger_original = tiger_original[:, :, ::-1]
plt.imshow(tiger_original)

#水平镜像
tiger_flip_Horizontally=cv2.flip(tiger_original,1)

#垂直镜像
tiger_flip_Vertically=cv2.flip(tiger_original,0)

#水平垂直镜像
tiger_flip_Horizontally_Vertically=cv2.flip(tiger_original,-1)

#保存图片
cv2.imwrite("./data/bird_flip_Horizontally.png",tiger_flip_Horizontally)
cv2.imwrite("./data/bird_flip_Vertically.png",tiger_flip_Vertically)
cv2.imwrite("./data/bird_flip_Horizontally_Vertically.png",tiger_flip_Horizontally_Vertically)
#-------- 请结束编写代码

## 7.旋转图片

OpenCV 中对图像的旋转主要是两步：
1. 先通过 getRotationMatrix2D 函数得到图像的旋转矩阵
2. 然后再通过仿射变换函数 warpAffine 得到旋转后的图像

> cv2.getRotationMatrix2D(center, angle, scale)  

> cv2.warpAffine(src, M, dsize,dst=None,flags=None,borderMode=None,borderValue=None)

-------
#### 参数说明：
**cv2.getRotationMatrix2D:**  
> - center：表示旋转的中心点  
> - angle：表示旋转的角度degrees  
> - scale：图像缩放因子  

**cv2.warpAffine:**  
> - src：输入的图像  
> - M：2 X 3 的变换矩阵  
> - dsize：输出的图像的size大小  
> - dst：输出的图像  
> - flags：输出图像的插值方法  
> - borderMode：图像边界的处理方式  
> - borderValue：当图像边界处理方式为BORDER_CONSTANT时的填充值  

In [21]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

img_original = cv2.imread('./data/back.jpg',1)
plt.imshow(img_original)

imgInfo = img_original.shape
height = imgInfo[0]
width = imgInfo[1]
print(imgInfo)

# 2*3 的旋转矩阵
matRotate = cv2.getRotationMatrix2D((height*0.5,width*0.5),45,0.5)# 缩放因子为0.5

img_rotation = cv2.warpAffine(img_original,matRotate,(height,width))

plt.imshow(img_rotation)

## 总结
本次实验主要使用 OpenCV 库完成了一些图片的操作。